In [ ]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np
import cv2
from pathlib import Path
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import time, gc
from math import ceil

## Data Params

In [ ]:
DATA_DIR = Path("../input/bengaliai-cv19/")
ORIGINAL_HEIGHT = 137
ORIGINAL_WIDTH = 236

## Hyperparameters

In [ ]:
WIDTH, HEIGHT = 250, 150
BATCH_SIZE = 64
EPOCHS = 5

In [ ]:
root_cls_num = 168
vowel_cls_num = 11
consonant_cls_num = 7

In [ ]:
class TestDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, df, batch_size = 16):
        self.df = df
        self.batch_size = batch_size
        self.image_ids = df["image_id"].values
        self.df = df.drop(["image_id"], axis=1)
                    
    def __len__(self):
        return int(ceil(len(self.df) / self.batch_size))

    def __getitem__(self, index):
        start_idx = index*self.batch_size
        batch_size = min(self.batch_size, len(self.df) - start_idx)
        X = np.empty((batch_size, *(HEIGHT, WIDTH, 3)))
        for idx in range(batch_size):
            idx = idx + start_idx
            flat_img = self.df.loc[df.index[idx]].values
            flat_img = 255 - flat_img
            flat_img = (flat_img * (255.0 / flat_img.max())).astype(np.uint8)
            img = flat_img.reshape((ORIGINAL_HEIGHT, ORIGINAL_WIDTH))
            img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
            img = cv2.resize(img, (WIDTH, HEIGHT))
            X[idx,] = img
        return X

In [ ]:
def create_model():
    inputs = tf.keras.Input(shape=(HEIGHT, WIDTH, 3), name='img')
    enet = tf.keras.applications.MobileNetV2(
        input_shape=(HEIGHT, WIDTH, 3),
        weights=None,
        include_top=False
    )
    x = enet(inputs)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    grapheme_root = tf.keras.layers.Dense(root_cls_num, activation='softmax', name='root')(x)
    vowel = tf.keras.layers.Dense(vowel_cls_num, activation='softmax', name='vowel')(x)
    consonant = tf.keras.layers.Dense(consonant_cls_num, activation='softmax', name='consonant')(x)
    
    model = tf.keras.Model(inputs=inputs, outputs=[grapheme_root, vowel, consonant])
    return model

In [ ]:
device = '/device:GPU:0'
num_test_parquetes = 4
row_ids = []
targets = [] 
with tf.device(device):
    model = create_model()
    model.load_weights("../input/bengaligraphememodels/cp.ckpt")
    for i in tqdm(range(num_test_parquetes)):
        df = pd.read_parquet(DATA_DIR/("test_image_data_" + str(i) + ".parquet"))
        test_data_generator = TestDataGenerator(df, batch_size=BATCH_SIZE)
        preds = model.predict(test_data_generator)
        for idx in range(preds[0].shape[0]):
            image_id = test_data_generator.image_ids[idx]
            row_ids.append(image_id + "_grapheme_root")
            row_ids.append(image_id + "_vowel_diacritic")
            row_ids.append(image_id + "_consonant_diacritic")
            root, vowel, consonant = np.argmax(preds[0][idx]), np.argmax(preds[1][idx]), np.argmax(preds[2][idx])
            targets.append(root)
            targets.append(vowel)
            targets.append(consonant)
        del df
        gc.collect()

In [ ]:
submit_df = pd.DataFrame({'row_id':row_ids,'target':targets}, columns = ['row_id','target'])
submit_df.to_csv('submission.csv', index = False)
print(submit_df.head(40))